# DQN Extensions

In [1]:
import pyvirtualdisplay, os
display = pyvirtualdisplay.Display(visible=0, size=(1400, 900)).start()

### 1. Basic DQN

In [2]:
import gym
import ptan
import random
import torch
import torch.optim as optim
from ignite.engine import Engine

In [3]:
from lib import dqn_model, common

In [4]:
NAME = "01_baseline"

In [5]:
params = common.HYPERPARAMS['pong'] # get hyperparams for pong
device = torch.device("cuda")

In [6]:
env = gym.make(params.env_name)
env = ptan.common.wrappers.wrap_dqn(env) # wrap common wrappers!

net = dqn_model.DQN(env.observation_space.shape, env.action_space.n).to(device)
tgt_net = ptan.agent.TargetNet(net)

In [7]:
selector = ptan.actions.EpsilonGreedyActionSelector(epsilon=params.epsilon_start) #this calculates the argmax and outputs the actions from probability
epsilon_tracker = common.EpsilonTracker(selector, params)
agent = ptan.agent.DQNAgent(net, selector, device=device)

In [8]:
exp_source = ptan.experience.ExperienceSourceFirstLast(env, agent, gamma=params.gamma, steps_count=1)
buffer = ptan.experience.ExperienceReplayBuffer(exp_source, buffer_size=params.replay_size)

In [9]:
optimizer = optim.Adam(net.parameters(), lr=params.learning_rate)

In [10]:
def process_batch(engine, batch): # THIS KEEPS ROLLING
    optimizer.zero_grad()
    loss_v = common.calc_loss_dqn(
        batch, net, tgt_net.target_model, gamma=params.gamma, device=device)
    loss_v.backward()
    optimizer.step()
    epsilon_tracker.frame(engine.state.iteration)
    if engine.state.iteration % params.target_net_sync == 0:
        tgt_net.sync()
    return {
        "loss": loss_v.item(),
        "epsilon": selector.epsilon,
    }

In [11]:
engine = Engine(process_batch) # ignite.engine
common.setup_ignite(engine, params, exp_source, NAME)

In [12]:
engine.run(common.batch_generator(buffer, params.replay_initial, params.batch_size))

Episode 1: reward=-20, steps=972, speed=0.0 f/s, elapsed=0:00:18
Episode 2: reward=-20, steps=970, speed=0.0 f/s, elapsed=0:00:18
Episode 3: reward=-21, steps=844, speed=0.0 f/s, elapsed=0:00:18
Episode 4: reward=-21, steps=761, speed=0.0 f/s, elapsed=0:00:18
Episode 5: reward=-21, steps=961, speed=0.0 f/s, elapsed=0:00:18
Episode 6: reward=-21, steps=846, speed=0.0 f/s, elapsed=0:00:18
Episode 7: reward=-21, steps=904, speed=0.0 f/s, elapsed=0:00:18
Episode 8: reward=-21, steps=846, speed=0.0 f/s, elapsed=0:00:18
Episode 9: reward=-20, steps=968, speed=0.0 f/s, elapsed=0:00:18
Episode 10: reward=-20, steps=913, speed=0.0 f/s, elapsed=0:00:18
Episode 11: reward=-21, steps=759, speed=0.0 f/s, elapsed=0:00:18
Episode 12: reward=-21, steps=845, speed=112.8 f/s, elapsed=0:00:23
Episode 13: reward=-21, steps=863, speed=112.7 f/s, elapsed=0:00:32
Episode 14: reward=-20, steps=958, speed=112.6 f/s, elapsed=0:00:40
Episode 15: reward=-20, steps=959, speed=112.6 f/s, elapsed=0:00:49
Episode 16:

State:
	iteration: 906059
	epoch: 1
	epoch_length: <class 'NoneType'>
	max_epochs: 1
	output: <class 'dict'>
	batch: <class 'list'>
	metrics: <class 'dict'>
	dataloader: <class 'generator'>
	seed: <class 'NoneType'>
	times: <class 'dict'>
	episode: 445
	episode_reward: 21.0
	episode_steps: 1634